In [1]:
import pandas as pd
import numpy as np
import statistics as stat
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("white")
import statistics as stat

In [2]:
data = pd.read_csv('scdb-200rfq-30cap-metrics.csv', names = ['Event', 'Time', 'Tx_type', 'Tx_id', 'Accept_id', 'time'])

In [3]:
grouped_data = data.groupby('Tx_id')
tx_id = list(grouped_data.groups.keys())
len(tx_id)

15329

In [4]:
#new data frame to get the count of each transaction type
data_unique_ids = data.drop_duplicates(subset = ['Tx_id'])

create_count = 0
create_ids = []
for i in range(0, len(data_unique_ids)):
    if data_unique_ids.iloc[i]['Tx_type'] == 'CREATE':
        create_ids.append(data_unique_ids.iloc[i]['Tx_id'])
        create_count += 1   


print(f"#CREATEs =  {create_count}")

#CREATEs =  4994


In [5]:
df_test = grouped_data.get_group("30dde698483f4f5c17186647a54cf21f2bfc745f4afb68a08bfeda867cbf7d1d")

KeyError: '30dde698483f4f5c17186647a54cf21f2bfc745f4afb68a08bfeda867cbf7d1d'

In [6]:
df_test

NameError: name 'df_test' is not defined

In [7]:
time_all_committed_creates = {}
time_passed_creates = []
validated_time = []
accepted_time = []
committed_time = []

create_underfit = []
create_underfit_dict = {}

create_overfit = []
create_overfit_dict = {}

for i in range(0, len(tx_id)):
    temp_list = []
    if "CREATE" == grouped_data.get_group(tx_id[i])["Tx_type"].to_list()[0]:
        temp_list = grouped_data.get_group(tx_id[i])["Time"].to_list()
        if len(temp_list) == 6 and grouped_data.get_group(tx_id[i])["Event"].to_list()[5] == "commit_tx":
            
            actual_time = max(temp_list) - min(temp_list)
            time_all_committed_creates[tx_id[i]] = actual_time
            time_passed_creates.append(actual_time)
            
            df = grouped_data.get_group(tx_id[i])
            received_time = df[df['Event'] == 'received_tx']['Time'].to_list()[0]
            before_tendermint_time = df[df['Event'] == 'before_tendermint']['Time'].to_list()[0]
            validated = before_tendermint_time - received_time
            validated_time.append(validated)
            check_time = df[df['Event'] == 'check_tx']['Time'].to_list()[0]
            accepted = check_time - received_time
            accepted_time.append(accepted)
            commit_time =  df[df['Event'] == 'commit_tx']['Time'].to_list()[0]
            committed = commit_time - received_time
            committed_time.append(committed)
        elif len(temp_list) < 6:
            create_underfit.append(temp_list)
            create_underfit_dict[tx_id(i)] = temp_list
        elif 7 >= len(temp_list) > 6 :
            df_check = grouped_data.get_group(tx_id[i])
            if len(df_check[df_check['Event'] == 'check_tx']['Time'].to_list()) > 1:
                check_time = max(df_check[df_check['Event'] == 'check_tx']['Time'].to_list())
            else:
                check_time = df_check[df_check['Event'] == 'check_tx']['Time'].to_list()[0] 
            
            actual_time = max(temp_list) - min(temp_list)
            time_all_committed_creates[tx_id[i]] = actual_time
            time_passed_creates.append(actual_time)
            
            df = grouped_data.get_group(tx_id[i])
            received_time = df[df['Event'] == 'received_tx']['Time'].to_list()[0]
            before_tendermint_time = df[df['Event'] == 'before_tendermint']['Time'].to_list()[0]
            
            validated = before_tendermint_time - received_time
            validated_time.append(validated)
           
            accepted = check_time - received_time
            accepted_time.append(accepted)
            
            commit_time =  df[df['Event'] == 'commit_tx']['Time'].to_list()[0]
            committed = commit_time - received_time
            committed_time.append(committed)
            create_overfit.append(temp_list)
            create_overfit_dict[tx_id[i]] = temp_list

In [8]:
# Analysis of CREATE Transaction
create_mean = "{:.2f}".format(stat.mean(time_passed_creates))
create_median = stat.median(time_passed_creates)
create_max = max(time_passed_creates)
create_min = min(time_passed_creates)
create_std_dev = "{:.2f}".format(stat.stdev(time_passed_creates))

print(create_mean)
print(create_median)
print(create_max)
print(create_min)
print(create_std_dev)
print(f"total creates = {len(time_passed_creates)}")

create_dict = {
               'min': create_min,
               'max': create_max,
               'average': create_mean,
               'median' : create_median,
               'standard_deviation' : create_std_dev,
               '# of txs' : len(time_passed_creates)}

82.20
68.0
768
62
40.28
total creates = 4994


In [9]:
create_overfit_dict

{'191215b96a8e78daf347db453c340d59ab7e3838b152c00e2cabda5ed6319026': [801,
  805,
  810,
  885,
  939,
  941,
  986],
 'f7b6e38b5638fbc3e860b269545c77527e92885bdec981977e6b88a2f5d2c978': [5666,
  5669,
  5679,
  5743,
  5799,
  5802,
  5888]}

In [10]:
time_all_committed_creates

{'00051a7b5c0a16bc16ccb8c824798e6d76d23a9af969f04bade54482fe474054': 66,
 '0006b09519ad2a20e5c104a0c5fde93701189449c615e795a3ac4bac2490b934': 97,
 '000ac07d7d7857e48545ee45e948fd00c9ef81ef338287a027e70f5cc1f99f97': 65,
 '000ef434afed891ce89c55db5ec860231e7a7d118669fffb5ee2d9266d1eee90': 68,
 '002a401be69f26dd96dd507e6bafac0167fb110b6ea305eae5df380f8d57dd4d': 66,
 '003298e656a5ecc27101db00eb7c768ccbc49aefc92d1e641327e490b8ae97c4': 66,
 '003305c8edd5b4df1245940f1c6e1f119eb2da472745031b405cd68fc73c2526': 66,
 '00439c69c060a9974373cf8bd506c919473e297e98f37ab08af53e31b986a6fa': 65,
 '004842700b794fbe7522281b7456515a7a2be95b4fecb090d4824f5cfd3299ed': 64,
 '004bc81e909351b821a81f362bc4898674908c930c5b0886458d9bd06edc3e26': 95,
 '00607b40766142b21a9077fc6aec7fbb2c9ee422da838018a63711132fd5a914': 87,
 '0066d8b6960ec9dabcdd160aeaffe64877bfcefd88a936f6326c4acc6fd65b24': 111,
 '0085467dde7012d415ad2e47caf8dd8e94400e76be6cff8dc6a1130d7edc3e2f': 68,
 '008fef9a097381416c5c5c472ed08a742cfe65f1bfca78d0

In [11]:
results = pd.DataFrame(create_dict, index = ['CREATE'])
results.to_csv('create_analysis.csv', sep=',', encoding='utf-8')
results

,min,max,average,median,standard_deviation,# of txs
CREATE,62,768,82.20,68.0,40.28,4994
